In [2]:
import os, sys

import sgpp, sgml, dproc, sgutil
import pandas as pd
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

print(sys.version)

from sklearn.pipeline import make_pipeline

3.12.6 (main, Sep 30 2024, 02:19:13) [GCC 9.4.0]


- Host_Popularity_percentage 범주형 변수화
  
- Guest_Popularity_percentage 범주형 변수화

- Host_Popularity_percentage 를 -1 ~ 1 사이로 MinMaxScaling 

- Number_of_Ads에 이상치가 존재합니다. 이를 최빈값으로 정리하고 0 ~ 1 사이로 MinMaxScaling

- Guest_Popularity_percentage 를 -1 ~ 1 사이로 MinMaxScaling

- Episode_Length_minutes 는 범주형 변수화

- Episode_Length_minutes를 5에서 120 사이로

In [3]:
sc = sgutil.SGCache('img', 'result', 'model')

In [4]:
p = make_pipeline(
    sgpp.PolarsProcessor(),
    sgpp.ExprProcessor({
        'ELm_num': (pl.col('Episode_Length_minutes').clip(5, 120) - 5) / 115,
        'GP': (pl.col('Guest_Popularity_percentage').fill_null(pl.col('Guest_Popularity_percentage').clip(0,100).mean()).clip(0, 100) / 100),
        'HP': pl.col('Host_Popularity_percentage').clip(0, 100) / 100,
        'NAd': (pl.when(pl.col('Number_of_Ads') > 4).then(0).otherwise(pl.col('Number_of_Ads'))).fill_null(0.0) /3 ,
        'Number_of_Ads': (pl.when(pl.col('Number_of_Ads') > 4).then(0).otherwise(pl.col('Number_of_Ads'))).fill_null(0).cast(pl.Int8),
        'ELm_na': pl.col('Episode_Length_minutes').is_null(),
        'GPp_na': pl.col('Guest_Popularity_percentage').is_null(),
        'ELm_sqrt': ((pl.col('Episode_Length_minutes').clip(5, 120) - 5) / 115)  ** 0.5
    }),
    sgpp.PandasConverter(index_col = 'id')
)
p.fit(['data/train.csv'])

Pipeline(steps=[('polarsprocessor',
                 <sgpp.PolarsProcessor object at 0x7f2474a8c500>),
                ('exprprocessor',
                 <sgpp.ExprProcessor object at 0x7f24747899d0>),
                ('pandasconverter',
                 <sgpp.PandasConverter object at 0x7f2474bf98e0>)])

In [5]:
df_train = p.transform(['data/train.csv'])
df_train_1 = df_train.loc[df_train['Episode_Length_minutes'].notna()]
target = 'Listening_Time_minutes'

In [6]:
import lightgbm as lgb
import xgboost as xgb
import catboost as cb
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import KFold, cross_validate, cross_val_score, ShuffleSplit
from sklearn.linear_model import LinearRegression
from mlxtend.feature_selection import SequentialFeatureSelector
kf = KFold(4, shuffle = True, random_state = 123)
ss = ShuffleSplit(1, train_size = 0.75, random_state = 123)

def get_validation_splitter(validation_fraction):
    return lambda x: train_test_split(x, test_size = validation_fraction)

def include_org(df, include_org = False):
    return pd.concat([df, df_org]) if include_org else df

config = {
    'predict_func': lambda m, df, X: pd.Series(m.predict(df[X]), index = df.index),
    'score_func': lambda df, prds: root_mean_squared_error(df[target], prds),
    'validation_splitter': get_validation_splitter,
    'progress_callback': sgml.ProgressCallBack(), 
    'return_train_scores': True,
    'train_data_proc': include_org,
    'y': target,
}

lr_adapter = sgml.SklearnAdapter(LinearRegression)

- Target Encoding 실험 재설계,
  > Episode_Length_minutes 제외한 모든 변수로 모든 데이터셋으로 Combination 1 ~ 4
  >
  > Episode_Length_minutes 포함 모든 변수로 Episode_Length_minutes 미결측 데이터셋으로 Combination 0 ~ 3

In [7]:
from itertools import combinations
from tqdm.notebook import tqdm 
from cuml.preprocessing import TargetEncoder

def get_tgt_rmse(df, X_tgt, smooth_space = np.linspace(1, 10, 10)):
    rmse_fold = list()
    best_prd = None
    best_rmse = np.inf
    for i in smooth_space:
        tgt = TargetEncoder(smooth = i, split_method ='continuous')
        prds = list()
        rmses = list()
        for train_idx, test_idx in kf.split(df[X_tgt], df[target]):
            df.iloc[train_idx].pipe(lambda x: tgt.fit(x[X_tgt], x[target]))
            df_valid = df.iloc[test_idx]
            prds.append(
                pd.Series(tgt.transform(df_valid[X_tgt]), index = df_valid.index)
            )
            rmses.append(
                root_mean_squared_error(df_valid[target], prds[-1])
            )
        rmse_fold.append(np.mean(rmses))
        if best_rmse > rmse_fold[-1]:
            best_rmse = rmse_fold[-1]
            best_prd = pd.concat(prds)
    return np.min(rmse_fold), smooth_space[np.argmin(rmse_fold)], best_prd

def get_tgt_rmse_list(df, X_values, smooth_space = [0.01, 0.1, 1, 10, 100, 1000]):
    results = list()
    best_prd_list = list()
    for i in tqdm(X_values):
        rmse, best_smooth, best_prd = get_tgt_rmse(df, i, smooth_space = smooth_space)
        results.append(
            pd.Series(
                [i, rmse, best_smooth], index = ['X_tgt', 'RMSE', 'smooth']
            )
        )
        best_prd_list.append(best_prd.rename('__'.join(i)))
    return pd.DataFrame(results).sort_values('RMSE'), pd.concat(best_prd_list, axis=1).sort_index()

In [8]:
X_val = [
    'Host_Popularity_percentage', 'Guest_Popularity_percentage', 'Number_of_Ads',
    'Episode_Sentiment', 'Episode_Title', 'Genre', 'Podcast_Name', 'Publication_Day', 'Publication_Time'
]

# ALL

In [9]:
df_c1, df_prd_1 = sc.cache_result(
    'tgt_rmse_c1_all', lambda : get_tgt_rmse_list(df_train, [list(i) for i in combinations(X_val, 1)]), rerun = 0
)
df_c1

  0%|          | 0/9 [00:00<?, ?it/s]

,X_tgt,RMSE,smooth
0,[Host_Popularity_percentage],26.851036,10
2,[Number_of_Ads],26.923613,10
1,[Guest_Popularity_percentage],26.991587,100
4,[Episode_Title],27.064105,100
6,[Podcast_Name],27.104318,100
3,[Episode_Sentiment],27.117177,100
8,[Publication_Time],27.130197,1000
5,[Genre],27.131325,1000
7,[Publication_Day],27.134080,1000


In [ ]:
df_c2, df_prd_2 = sc.cache_result(
    'tgt_rmse_c2_all', lambda : get_tgt_rmse_list(df_train, [list(i) for i in combinations(X_val, 2)]), rerun = 0
)
df_c2.iloc[:25]

In [ ]:
df_c3, df_prd_3 = sc.cache_result(
    'tgt_rmse_c3_all', lambda : get_tgt_rmse_list(df_train, [list(i) for i in combinations(X_val, 3)]), rerun = 0
)
df_c3.iloc[:25]

In [ ]:
df_c4, df_prd_4 = sc.cache_result(
    'tgt_rmse_c4_all', lambda : get_tgt_rmse_list(df_train, [list(i) for i in combinations(X_val, 4)]), rerun = 0
)
df_c4.iloc[:25]

In [ ]:
df_c5, df_prd_5 = sc.cache_result(
    'tgt_rmse_c5_all', lambda : get_tgt_rmse_list(df_train, [list(i) for i in combinations(X_val, 5)]), rerun = 0
)
df_c5.iloc[:25]

# Episode_Length_minutes Not NA

In [ ]:
df_c0_2, df_prd_0_2 = sc.cache_result(
    'tgt_rmse_c0_2', lambda : get_tgt_rmse_list(df_train, [['Episode_Length_minutes']]), rerun = 0
)
df_c0_2

In [ ]:
df_c1_2, df_prd_1_2 = sc.cache_result(
    'tgt_rmse_c1_2', 
    lambda : get_tgt_rmse_list(df_train, [['Episode_Length_minutes'] + list(i) for i in combinations(X_val, 1)]), rerun = 0
)
df_c1_2

In [ ]:
df_c2_2, df_prd_2_2 = sc.cache_result(
    'tgt_rmse_c2_2', 
    lambda : get_tgt_rmse_list(df_train, [['Episode_Length_minutes'] + list(i) for i in combinations(X_val, 2)]), rerun = 0
)
df_c2_2

In [ ]:
df_c3_2, df_prd_3_2 = sc.cache_result(
    'tgt_rmse_c3_2', 
    lambda : get_tgt_rmse_list(df_train, [['Episode_Length_minutes'] + list(i) for i in combinations(X_val, 3)]), rerun = 0
)
df_c3_2

In [ ]:
df_c4_2, df_prd_4_2 = sc.cache_result(
    'tgt_rmse_c4_2', 
    lambda : get_tgt_rmse_list(df_train, [['Episode_Length_minutes'] + list(i) for i in combinations(X_val, 4)]), rerun = 0
)
df_c4_2

# Linear Regression

In [ ]:
result_list, tgt_list = list(), list()
for i in ['tgt_rmse_c1_all', 'tgt_rmse_c2_all', 'tgt_rmse_c3_all', 'tgt_rmse_c4_all', 
          'tgt_rmse_c0_2', 'tgt_rmse_c1_2', 'tgt_rmse_c2_2', 'tgt_rmse_c3_2']:
    df_result, df_tgt = sc.read_result(i)
    result_list.append(df_result)
    tgt_list.append(df_tgt)
df_tgt, df_tgt_result = pd.concat(tgt_list, axis=1).join(df_train[target]), pd.concat(result_list)
del result_list, tgt_list

In [ ]:
from itertools import combinations
X_val = [
    'Host_Popularity_percentage', 'Guest_Popularity_percentage', 'Number_of_Ads',
    'Episode_Sentiment', 'Episode_Title', 'Genre', 'Podcast_Name', 'Publication_Day', 'Publication_Time'
]
X_tgts = {
    **{'tgt_{}'.format(j): ['__'.join(i) for i in combinations(X_val, j)] for j in range(1, 5)},
    **{'tgt_{}_2'.format(j): ['__'.join(['Episode_Length_minutes'] + list(i)) for i in combinations(X_val, j)] for j in range(0,4)}
}

## ALL

In [ ]:
from sklearn.linear_model import LinearRegression

hparams = {
    'X_num': X_tgts['tgt_1'] + X_tgts['tgt_2'] + X_tgts['tgt_3'] + X_tgts['tgt_4']
}

result_1 = sc.cv_result('LR_0', df_tgt, kf, hparams, config, lr_adapter, result_proc = [sgml.lr_learning_result])
np.mean(result_1['valid_scores'])

In [ ]:
pd.DataFrame([i['coef'] for i in result['model_result']]).abs().mean().sort_values(ascending = False).iloc[:25]

### Step-wise Forward Feature Selection

지나치게 오래걸립니다.

In [ ]:
"""
sfs = SequentialFeatureSelector(
    LinearRegression(), k_features = 'best', forward = True, floating = True, scoring = 'neg_root_mean_squared_error', cv = kf, 
    fixed_features = ['Host_Popularity_percentage__Guest_Popularity_percentage']
)
"""

### PCA + Sequential Addition

In [ ]:
from sklearn.decomposition import PCA
from tqdm.notebook import tqdm
pca = PCA()
pca.fit(df_tgt)

In [ ]:
pca.explained_variance_ratio_.cumsum()
X_tgt_pca = pca.transform(df_tgt)

In [ ]:
scores = list()
for i in tqdm(list(range(2, X_tgt_pca.shape[1] + 1))):
    scores.append(
        np.mean(cross_val_score(LinearRegression(), X_tgt_pca[:, :i], df_train[target], cv = kf, scoring = 'neg_root_mean_squared_error'))
    )

In [ ]:
np.argmax(scores), np.max(scores), scores[-1], pca.explained_variance_ratio_[:(np.argmax(scores) - 1)].sum()

- 거의 모든 속성이 유의미 하다고 보여집니다.

# Episode_Length_minutes Not NA

In [ ]:
from sklearn.linear_model import LinearRegression

hparams = {
    'X_num': X_tgts['tgt_1'] + X_tgts['tgt_2'] + X_tgts['tgt_3'] + X_tgts['tgt_4'] + \
            X_tgts['tgt_0_2'] + X_tgts['tgt_1_2'] + X_tgts['tgt_2_2'] + X_tgts['tgt_3_2']
}

result_2 = sc.cv_result('LR_0_2', df_tgt.loc[~df_train['ELm_na']], kf, hparams, config, lr_adapter, result_proc = [sgml.lr_learning_result])
np.mean(result_2['valid_scores'])

In [ ]:
pd.DataFrame([i['coef'] for i in result['model_result']]).abs().mean().sort_values(ascending = False).iloc[:25]

In [ ]:
(
    (
        (np.mean(result_1['valid_scores']) ** 2) * df_train['ELm_na'].sum() + (np.mean(result_2['valid_scores']) ** 2) * (~df_train['ELm_na']).sum()
    ) / df_train.shape[0]
) ** 0.5

## ALL with all tgt values

In [ ]:
hparams = {
    'X_num': X_tgts['tgt_1'] + X_tgts['tgt_2'] + X_tgts['tgt_3'] + X_tgts['tgt_4'] +\
            X_tgts['tgt_0_2'] + X_tgts['tgt_1_2'] + X_tgts['tgt_2_2'] + X_tgts['tgt_3_2']
}

result_3 = sc.cv_result('LR_0_3', df_tgt, kf, hparams, config, lr_adapter, result_proc = [sgml.lr_learning_result])
result_3['valid_scores'], np.mean(result_3['valid_scores'])

- Episode_Length_minutes이 NA일 때외 NA가 아닐 때 따로 학습시켰을 때 효과가 더 좋습니다.


# Generating Target Encoded Data for test set

유의한 Target Encoding 결과를 가려내는 것은 지금 상태로썬 소모적으로 보입니다.

또한 Episode_Length_minutes 와 Combination 4는 들어가는 리소스 대비 성능향상이 너무 미약하여 제외 합니다.

In [ ]:
df_test = p.transform(['data/test.csv'])

In [ ]:
X_tgt_list = list()
for i in range(1, 6):
    X_tgt_list.extend([list(i) for i in combinations(X_val, i)])
X_tgt_list.append(['Episode_Length_minutes'])
for i in range(1, 4):
    X_tgt_list.extend([['Episode_Length_minutes'] + list(i) for i in combinations(X_val, i)])

In [ ]:
def get_tgt_fit_transform(df_train, df_test, X_tgt, smooth):
    results = list()
    best_prd_list = list()
    tgt = TargetEncoder(smooth = i, split_method ='continuous')
    return pd.Series(
        tgt.fit(df_train[X_tgt], df_train[target]).transform(df_test[X_tgt]),
        index = df_test.index, name = '__'.join(X_tgt)
    )

In [ ]:
s_smooth = pd.concat([
    df_c0_2, df_c1_2, df_c2_2, df_c3_2, df_c4_2, df_c1, df_c2, df_c3, df_c4, df_c5
], axis=0).assign(
    X_tgt = lambda x: x['X_tgt'].apply(lambda x: '__'.join(x) if type(x) == list else x)
).set_index('X_tgt')['smooth']

In [ ]:
df_tgt_test = sc.cache_result(
    'tgt_test',
    lambda : pd.concat([
        get_tgt_fit_transform(df_train, df_test, i, s_smooth['__'.join(i)])
        for i in X_tgt_list
    ], axis= 1), rerun = 1
)

In [ ]:
df_tgt = pd.concat([df_prd_1, df_prd_2, df_prd_3, df_prd_4, df_prd_5], axis=1)
df_tgt.head()

# TGT406 + LR

In [ ]:
X_lr = df_tgt.columns.tolist()
reg_lr = LinearRegression().fit(df_tgt, df_train[target])

In [ ]:
s_prd1 = df_tgt_test.loc[df_test['Episode_Length_minutes'].isna(), X_lr].pipe(
    lambda x: pd.Series(reg_lr.predict(x), index = x.index, name = target)
)
s_prd1.head()

In [ ]:
df_tgt = pd.concat([df_prd_0_2, df_prd_1_2, df_prd_2_2, df_prd_3_2, df_prd_1, df_prd_2, df_prd_3, df_prd_4, df_prd_5], axis=1).loc[
    df_train_1.index
]
df_tgt.head()

In [ ]:
X_lr = df_tgt.columns.tolist()
reg_lr = LinearRegression().fit(df_tgt, df_train_1[target])

In [ ]:
s_prd2 = df_tgt_test.loc[df_test['Episode_Length_minutes'].notna(), X_lr].pipe(
    lambda x: pd.Series(reg_lr.predict(x), index = x.index, name = target)
)
s_prd2.head()

In [ ]:
pd.concat([s_prd1, s_prd2]).sort_index().to_csv('result/submission1.csv')

# TGT406 + MM Scaled mean imputed X_num + LR

In [ ]:
hparams = {
    'X_ohe': ['Podcast_Name', 'Episode_Title', 'Genre', 'Publication_Day', 'Publication_Time', 'Episode_Sentiment'],
    'ohe': {'drop': 'first'},
    'X_num': ['GP', 'HP', 'NAd']
}

result = sgml.cv(df_train, kf, hparams, config, lr_adapter, result_proc = [sgml.lr_learning_result])
np.mean(result['valid_scores'])

In [ ]:
df_train[target].std()

In [ ]:
hparams = {
    'X_num': ['GP', 'HP', 'NAd'] + X_tgts['tgt_1'] + X_tgts['tgt_2'] + X_tgts['tgt_3'] + X_tgts['tgt_4']
}

result_1 = sgml.cv(
    df_train.drop(columns = [
        'Podcast_Name', 'Episode_Title', 'Episode_Length_minutes', 'Genre',
        'Host_Popularity_percentage', 'Publication_Day', 'Publication_Time',
        'Guest_Popularity_percentage', 'Number_of_Ads', 'Episode_Sentiment', 'Listening_Time_minutes'
    ]).join(df_tgt), kf, hparams, config, lr_adapter, result_proc = [sgml.lr_learning_result]
)
np.mean(result_1['valid_scores'])

In [ ]:
hparams = {
    'X_num': ['GP', 'HP', 'NAd', 'ELm_num', 'ELm_sqrt'] + X_tgts['tgt_1'] + X_tgts['tgt_2'] + X_tgts['tgt_3'] + X_tgts['tgt_4'] + X_tgts['tgt_0_2'] + X_tgts['tgt_1_2'] + X_tgts['tgt_2_2'] + X_tgts['tgt_3_2']
}

result_2 = sgml.cv(
    df_train.drop(columns = [
        'Podcast_Name', 'Episode_Title', 'Episode_Length_minutes', 'Genre',
        'Host_Popularity_percentage', 'Publication_Day', 'Publication_Time',
        'Guest_Popularity_percentage', 'Number_of_Ads', 'Episode_Sentiment', 'Listening_Time_minutes'
    ]).join(df_tgt).loc[~df_train['ELm_na']], kf, hparams, config, lr_adapter, result_proc = [sgml.lr_learning_result]
)
np.mean(result_2['valid_scores'])

In [ ]:
(
    (
        (np.mean(result_1['valid_scores']) ** 2) * df_train['ELm_na'].sum() + (np.mean(result_2['valid_scores']) ** 2) * (~df_train['ELm_na']).sum()
    ) / df_train.shape[0]
) ** 0.5

- Target Encoding 변수와 Episode_Length_minutes를 같이 사용하게 되면 오히려 정확도가 떨어집니다.